In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/eliaszachary@bcp.com.pe/Modulo de Seguimiento/')

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

# Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

## Base de seguimiento

In [0]:
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim
from pyspark import StorageLevel

import warnings

# Omitir todas las advertencias
warnings.filterwarnings("ignore")

# Parámetros de guardado
vUsuario = 'T70431'
vP = 'Em1z4marrod'
vNumReg = 10000000
vStorage = "dtbrcu1lhclp02"
vRutaEdvDestino = "bcp-edv-fabseg/T70431/Bases DWH/Base Applicant Hipotecario"
vContainer = vRutaEdvDestino.split("/", 1)[0]
vRutaEDV = vRutaEdvDestino[len(vContainer) : len(vRutaEdvDestino)]
vQueryOrigen = "SELECT * FROM S35105.HM_MTZ_ADM_HIPOTECARIO_DRIVER WHERE CODMES >= 201601"

try:
    # Lectura de la base de hipotecario
    base_app_veh = (
        spark.read.format("jdbc")
        .option("url", "jdbc:oracle:thin:@PRODREG.credito.bcp.com.pe:1521/BCPDW3")       
        .option("dbtable", "(SELECT ROWNUM as R, t.* FROM ("+vQueryOrigen+") t ) mitabla")
        .option("fetchSize",100000) 
        .option("partitionColumn", "R")
        .option("lowerBound", "1")
        .option("upperBound", vNumReg)
        .option("numPartitions", "50")
        .option("user", vUsuario)
        .option("password", vP)
        .option("driver", "oracle.jdbc.driver.OracleDriver")
        .option("oracle.jdbc.timezoneAsRegion", "false")
        .option("oracle.jdbc.defaultCharSet", "UTF-8")
        .load()
    )
    print(f"Conexión exitosa: consulta '{vQueryOrigen}' leida correctamente.")

    # Guardar el DataFrame en un contenedor de Azure Storage Account
    base_app_veh.write.format("delta").mode("overwrite").save("abfss://"+vContainer+"@"+vStorage+".dfs.core.windows.net"+vRutaEDV)

    spark.sql(
    '''
        DROP TABLE IF EXISTS catalog_lhcl_prod_bcp.bcp_edv_fabseg.T70431_BASE_APP_HIP
    '''
    )
    spark.sql('''
        CREATE TABLE 
            catalog_lhcl_prod_bcp.bcp_edv_fabseg.T70431_BASE_APP_HIP
        USING DELTA LOCATION 
            'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T70431/Bases DWH/Base Applicant Hipotecario' 
    '''
    )

except Exception as e:
    print(f"Conexión fallida: {e}")


        #    'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T70431/Bases DWH/Base Applicant Hipotecario' colocar nueva ruta de azure
